In [6]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv(
    r"C:\Users\WIN\OneDrive\Desktop\WiDS 5.0\master_dataset_final.csv"
)

count_vectorizer = CountVectorizer(
    analyzer='word',
    ngram_range=(1, 2),
    min_df=5,
    stop_words='english'
)

count_matrix = count_vectorizer.fit_transform(df['soup'])

cosine_sim = cosine_similarity(count_matrix, count_matrix)

cosine_sim_df = pd.DataFrame(
    cosine_sim,
    index=df['title'],
    columns=df['title']
)

cosine_sim_df.to_csv(
    r"C:\Users\WIN\OneDrive\Desktop\WiDS 5.0\cosine_similarity_matrix.csv"
)

def recommend_movies(movie_title, top_n=10, min_similarity=0.15):
    
    if movie_title not in df['title'].values:
        return "Movie not found in dataset"

    idx = df[df['title'] == movie_title].index[0]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    recommendations = []

    for i, score in sim_scores[1:]:
        if score < min_similarity:
            break

        recommendations.append({
            "Movie Title": df.iloc[i]['title'],
            "Director": df.iloc[i]['main_director'],
            "Release Date": df.iloc[i]['release_date'],
            "Similarity Score": round(score, 3)
        })

        if len(recommendations) == top_n:
            break

    return pd.DataFrame(recommendations)


recommend_movies("Avatar", top_n=8, min_similarity=0.15)



,Movie Title,Director,Release Date,Similarity Score
0,Aliens,James Cameron,1986-07-18,0.651
1,Titanic,James Cameron,1997-11-18,0.513
2,True Lies,James Cameron,1994-07-14,0.498
3,Terminator 2: Judgment Day,James Cameron,1991-07-01,0.487
4,The Terminator,James Cameron,1984-10-26,0.467
5,Star Trek Into Darkness,J.J. Abrams,2013-05-05,0.216
6,Home,Tim Johnson,2015-03-18,0.191
7,Behind Enemy Lines II: Axis of Evil,James Dodson,2006-10-17,0.180
